In [2]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [3]:
import sys
sys.path.insert(0, MAIN_PATH)

In [15]:
%load_ext autoreload
%autoreload 2
from scripts.models.rank_with_model.rank_with_model import RankModel
from scripts.models.ltr.train import LTRTrain
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Learning to Rank: salida ordenada

Una vez se ha obtenido un score que permita ordenar cada evento dentro
de un partido, se debe construir el "resumen" a partir de ello.

La aproximación más sencilla sería ordenar los eventos para cada partido, y escoger los N primeros eventos con mayor score para representar la noticia. Sin embargo, como ya hemos comentado con 
anterioridad, esto puede sufrir varios problemas: puede haber muchos
eventos con el mismo score, y además muchos eventos pueden estar
referidos a la misma frase del artículo.

# Uso de modelos

Una vez obtenido un baseline usando directamente los scores, se 
pasa a utilizar la predicción de estos scores para ordenar los eventos, usando los modelos entrenados.

## Random forest

In [16]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'rouge',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
cat_features_dict = {'is_key_event': [0, 1]}
num_features = ['tfidf_sum']
model_params = {
    'n_estimators': 100,
    'max_depth': 2,
    'bootstrap': True,  
}

In [17]:
ltr_train = LTRTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params)

Setting target metric to rouge


In [18]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [19]:
rank_model.run()

Categorizing n_players...
Computing new length...
Dropping {'n_players_cat', 'url', 'n_stop', 'score', 'sim_previous_5', 'sim_previous_3', 'sim_previous_1', 'sentence_ix', 'players_importance', 'event_ix', 'length', 'advantage', 'json_file', 'position', 'n_players', 'equalize', 'total_length'}
Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/9ef9119130/summaries.csv
